In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import re

In [2]:
# GATHERING NAMES DATA

rusn = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/dict_names/russian_names.csv', encoding = 'utf8',\
                 delimiter = ';')
forn = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/dict_names/foreign_names.csv', encoding = 'utf8',\
                 delimiter = ';')
rusn = rusn[['Name', 'Sex', 'Source']]
forn = forn[['name', 'gender','Source']]
rusn.columns = ['dict_first_name', 'dict_sex', 'Source']
forn.columns = ['dict_first_name', 'dict_sex', 'Source']
forn['dict_sex'] = forn['dict_sex'].replace('Male', 'М')
forn['dict_sex'] = forn['dict_sex'].replace('Female', 'Ж')
names_dict = pd.concat([rusn, forn]).reset_index(drop = True)

In [3]:
# DATA QUALITY ANALYSIS WITH TWO TYPES OF ERRORS: NA AND ERROR

# - ПРОВЕРИТЬ ПРОТУХШИЕ ДОКИ
# - СКВОЗНЫЕ ДАННЫЕ КОТОРЫЕ МОЖНО ПРОВЕРИТЬ ПО ДРУГИМ ДОКАМ (ФИО ПО ПАСПОРТУ И ВУ)
# - МОРФОЛОГИЧЕСКИЕ ОШИБКИ В ДАННЫХ КОТОРЫЕ МОЖНО ПРОВЕРИТЬ ЧЕРЕЗ API DADATA ИЛИ ЯНДЕКСА
# Detailed passport (fields not required for KBM) and driving license data quality analysis
df = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/original_data/users_last_ride_during_Q1_2020.csv', low_memory = False)
# СОРТИРОВКА РЕГИСТРАЦИИ С СЕНТЯБРЯ!!!
# СОРТИРОВКА РЕГИСТРАЦИИ С СЕНТЯБРЯ!!!
# СОРТИРОВКА РЕГИСТРАЦИИ С СЕНТЯБРЯ!!!
# df = df.loc[(df['NameRu'] == 'Активирован')&(df.registration_dtime > '2019-08-31')]
df = pd.merge(df, names_dict, left_on = 'first_name', right_on ='dict_first_name', how = 'left')
# Format datetime data
df['birthday_dt'] = pd.to_datetime(df['birthday'], errors = 'coerce')
df['license_set_dt'] = pd.to_datetime(df['license_set_date'], errors = 'coerce')
# Format docs image availability data
df['passport'] = df['passport'].fillna(False)
df['drivers'] = df['drivers'].fillna(False)
df['selfie'] = df['selfie'].fillna(False)
# Marking null values
for i in ['license_set_dt', 'License', 'PassportNumber', 'PassportDepartmentCode', 'PassportDepartment', 'PassportRegistration',\
          'birth_place', 'birthday_dt', 'sex', 'first_name', 'patronymic_name', 'last_name']:
    if i == 'first_name' or i =='patronymic_name' or i =='last_name':
        df[i+'_null'] = (pd.isnull(df[i]))|\
                        (df[i] == 'Нет')|\
                        (df[i] == 'нет')|\
                        (df[i] == 'Отсутствует')|\
                        (df[i] == 'отсутствует')|\
                        (df[i] == 'В')|\
                        (df[i] == 'Процессе')|\
                        (df[i] == 'Регистрации')
    else:
        df[i+'_null'] = (pd.isnull(df[i]))        
# Marking incorrect datetime data
df['lcns_incorrect_dt'] = (pd.to_datetime('today').year - df.license_set_dt.dt.year < 1.0)|\
                              (pd.to_datetime('today').year - df.license_set_dt.dt.year > 64.0)|\
                              (df.license_set_dt.dt.year-df.birthday_dt.dt.year < 16.0)
df['bd_incorrect_dt'] = ((pd.to_datetime('today')-df['birthday_dt']).astype('timedelta64[Y]') < 19.0)|\
                        ((pd.to_datetime('today')-df['birthday_dt']).astype('timedelta64[Y]') > 80.0)
# Marking incorrect pattern/length
for i in ['License', 'PassportNumber']:
    df[i+'_incorrect_len'] = (df[i].str.replace(' ','')).str.len()!=10.0
    df[i+'_incorrect_ptrn'] = (df[i].str.replace(' ','')).str.contains(pat = '\w{10}') == False
df['psp_dep_code_incorrect_len'] = df['PassportDepartmentCode'].str.len()!=7.0
df['psp_dep_code_incorrect_ptrn'] = df['PassportDepartmentCode'].str.contains(pat = '\d\d\d-\d\d\d') == False
# Marking errors with name
for i in ['first_name', 'patronymic_name', 'last_name']:
    df[i+'_nick'] = df[i].str.contains('Мойка')|\
                    df[i].str.contains('Каршайн')|\
                    df[i].str.contains('Карсервис')|\
                    df[i].str.contains('Омывайка')|\
                    df[i].str.contains('Нафта')|\
                    df[i].str.contains('Пройл')|\
                    df[i].str.contains('Пролив')|\
                    df[i].str.contains('Механик')|\
                    df[i].str.contains('Спб')|\
                    df[i].str.contains('\(')|\
                    df[i].str.contains('\)')
    df[i+'_format'] = (df[i].str.contains(pat = '[a-zA-Z]', regex = True))|\
                      (df[i].str.isalpha() == False)
df['first_name_grammar'] = pd.isnull(df.Source) == True

# Marking errors in name
name_cols = ['first_name_null','patronymic_name_null','last_name_null','first_name_nick','first_name_format','patronymic_name_nick','patronymic_name_format','last_name_nick',\
             'last_name_format', 'first_name_grammar']
df['FIO_error'] = df[name_cols].sum(axis = 1)>0

# Marking errors in passport
psp_cols = ['first_name_null','patronymic_name_null','last_name_null','first_name_nick','first_name_format',\
            'patronymic_name_nick','patronymic_name_format','last_name_nick','last_name_format','PassportNumber_null',\
            'PassportDepartmentCode_null','PassportDepartment_null','PassportRegistration_null',\
            'birth_place_null','sex_null','birthday_dt_null','PassportNumber_incorrect_len','PassportNumber_incorrect_ptrn',\
            'psp_dep_code_incorrect_len','psp_dep_code_incorrect_ptrn','bd_incorrect_dt']
df['passport_error'] = df[psp_cols].sum(axis = 1)>0

# Marking errors in driving license
lcns_cols = ['License_null', 'license_set_dt_null','License_incorrect_len','License_incorrect_ptrn','lcns_incorrect_dt']
df['driving_license_error'] = df[lcns_cols].sum(axis = 1)>0

# Marking errors in driving license number
lcns_nmbr_cols = ['License_null', 'License_incorrect_len','License_incorrect_ptrn']
df['driving_license_number_error'] = df[lcns_nmbr_cols].sum(axis = 1)>0

# Marking errors in driving license set date
lcns_set_date_cols = ['license_set_dt_null','lcns_incorrect_dt']
df['driving_license_set_date_error'] = df[lcns_set_date_cols].sum(axis = 1)>0

# Marking errors in fields used for KBM request
KBM_cols = ['first_name_null','patronymic_name_null','last_name_null','birthday_dt_null','License_null','sex_null',\
            'first_name_nick','first_name_format','patronymic_name_nick','patronymic_name_format', 'last_name_nick',\
            'last_name_format','first_name_grammar','bd_incorrect_dt','License_incorrect_len','License_incorrect_ptrn']
df['KBM_cols_error'] = df[KBM_cols].sum(axis = 1)>0

# Marking errors in birthday date
bd_cols = ['birthday_dt_null','bd_incorrect_dt']
df['bd_error'] = df[bd_cols].sum(axis = 1)>0

# Marking errors in fields used for scoring
scoring_cols = ['sex_null', 'birthday_dt_null','license_set_dt_null','birth_place_null','bd_incorrect_dt','lcns_incorrect_dt']
df['scoring_cols_error'] = df[scoring_cols].sum(axis = 1)>0

# Marking rows with any errors
cols = ['first_name_null','patronymic_name_null','last_name_null','sex_null','birthday_dt_null','License_null',\
        'license_set_dt_null','PassportNumber_null','PassportDepartmentCode_null','PassportDepartment_null',\
        'PassportRegistration_null','birth_place_null','first_name_nick','first_name_format','patronymic_name_nick',\
        'patronymic_name_format','last_name_nick', 'last_name_format', 'bd_incorrect_dt', 'License_incorrect_len',\
        'License_incorrect_ptrn','lcns_incorrect_dt','PassportNumber_incorrect_len','PassportNumber_incorrect_ptrn',\
        'psp_dep_code_incorrect_len','psp_dep_code_incorrect_ptrn']
aggs = ['FIO_error','passport_error', 'driving_license_error',\
        'driving_license_number_error','driving_license_set_date_error',\
        'KBM_cols_error','bd_error','scoring_cols_error',\
        'any_error']
df['any_error'] = df[cols].sum(axis = 1)>0

# Leaving only users that are Russian citizens
df_rus = df.loc[df.rus_ctzn == 1.0]

# Creating dataframe with error stats
# stats = df_rus['license_set_dt_null'].value_counts().rename_axis('error').reset_index(name='license_set_dt_null')
# for i in cols+aggs:
#     stats[i] = df_rus[i].value_counts()
# stats.to_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/data_quality_results/error_stats/data_quality_results_active_users.csv')

In [4]:
# rus = df_rus.loc[(df_rus.KBM_cols_error == True)|(df_rus.birth_place_null == True)]
# total = df.loc[(df.KBM_cols_error == True)|(df.birth_place_null == True)]
# total
df

,user_id,user_ext,login,sumsub_applicant_id,NameRu,activation_dtime,registration_dtime,License,license_set_date,first_name,...,first_name_grammar,FIO_error,passport_error,driving_license_error,driving_license_number_error,driving_license_set_date_error,KBM_cols_error,bd_error,scoring_cols_error,any_error
0,19500003,33,79030926325,5e83a1852061112027db65c0,Активирован,2015-09-15 16:26:22,2015-07-10 00:00:00,7708214084,2013-01-01,Михаил,...,False,False,False,False,False,False,False,False,False,False
1,19500013,93,79853423032,5d5514c30a975a20be6968fe,Активирован,2015-07-15 10:19:01,2015-07-15 00:00:00,9911 098329,2000-01-01,Карен,...,False,False,False,False,False,False,False,False,False,False
2,19500025,137,79853697656,5e84c1aed5ea4874e2e1a5fa,Активирован,2015-09-19 00:00:00,2015-09-19 00:00:00,21 15 263819,2013-01-01,Алексей,...,False,False,False,False,False,False,False,False,False,False
3,19500028,147,79267904212,NaN,Активирован,2015-07-20 19:17:55,2015-07-20 00:00:00,77УЕ393854,NaN,Давид,...,False,False,False,True,False,True,False,False,True,True
4,19500035,171,79165077513,NaN,На проверке,2015-07-22 20:04:16,2015-07-22 00:00:00,77УЕ060390,1998-01-01,Константин,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654239,1252000030,30684280,79179993429,5ec26e0953d4866316f419b2,Активирован,2020-05-18 14:19:03,2020-05-18 07:38:56,1318 482221,2014-01-01,Vladislav,...,True,True,True,False,False,False,True,False,False,True
654240,1252000037,30684303,79531860505,5ec21b8abb03f104387c10fe,Активирован,2020-05-18 08:36:03,2020-05-18 07:51:30,7117 216300,2007-01-01,Антон,...,False,False,True,False,False,False,False,False,False,True
654241,1252500275,30686970,79024139674,5ec2d4aa53d4866316f9e793,Активирован,2020-05-18 21:43:02,2020-05-18 21:12:51,9902 905993,2018-01-01,Радик,...,False,False,True,False,False,False,False,False,False,True
654242,1252750047,30685230,79159303360,5ec2663253d4866316f3a80f,Активирован,2020-05-18 13:45:02,2020-05-18 13:34:53,5223 209303,2015-01-01,Александр,...,False,False,True,False,False,False,False,False,False,True


In [9]:
batch1 = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/data_quality_results/all_docs/batch_#1/no_S&S_check_v4.csv')
expired_docs = pd.read_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/data_quality_results/all_docs/users_w_expired_docs.csv')
batch1['batch1'] = 1
batch1 = batch1[['user_id', 'batch1']]
expired_docs = expired_docs[['user_id', 'passport_expired','drivers_license_expired']]
batch2 = df_rus.loc[(df_rus['KBM_cols_error'] == True)|(df_rus['birth_place_null'] == True)]
batch2 = pd.merge(pd.merge(batch2, batch1, on = 'user_id', how = 'left'),expired_docs, on = 'user_id', how = 'left')
batch2 = batch2.loc[(pd.isnull(batch2.batch1) == True)&(batch2.passport_expired != True)&(batch2.drivers_license_expired != True)]
batch2['urgency_rank'] = np.where(batch2.KBM_cols_error == True, 'urgency_#1', 'urgency_#2')
batch2
# batch2.to_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/data_quality_results/all_docs/batch_#2/batch_#2.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,user_ext,login,sumsub_applicant_id,NameRu,activation_dtime,registration_dtime,License,license_set_date,first_name,...,driving_license_number_error,driving_license_set_date_error,KBM_cols_error,bd_error,scoring_cols_error,any_error,batch1,passport_expired,drivers_license_expired,urgency_rank
2,19500284,1443,79067273993,5b2817c30a975a0414053938,Активирован,2015-08-12 13:08:39,2015-08-12 00:00:00,77 08 263954,2012-01-01,Сергей,...,False,False,False,False,True,True,NaN,NaN,NaN,urgency_#2
3,19500400,2058,79031297777,5bb0e9c40a975a445419dc02,Активирован,2019-07-21 17:16:07,2015-08-18 00:00:00,9902 385351,2011-01-01,Евгений,...,False,False,False,False,True,True,NaN,NaN,NaN,urgency_#2
4,19500434,2177,79265849171,5b6db1fc0a975a5b95f82c26,Активирован,2015-08-19 12:42:45,2015-08-19 00:00:00,77 06 450994,2011-01-01,Оганес,...,False,False,False,False,True,True,NaN,NaN,NaN,urgency_#2
7,19500694,3326,79165558121,5b7d8e3d0a975a328e3d9d76,Активирован,2015-09-05 20:53:17,2015-09-05 00:00:00,50ОТ655281,1999-01-01,Александр,...,False,False,False,False,True,True,NaN,NaN,NaN,urgency_#2
8,19500883,4140,79037552632,5b60d9180a975a196379779a,Активирован,2015-09-10 13:19:10,2015-09-10 00:00:00,99 02 324159,2007-01-01,Александр,...,False,False,False,False,True,True,NaN,NaN,NaN,urgency_#2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172839,1221750082,30644915,79632610303,5eb29058bb03f1650ead2c15,Активирован,2020-05-06 13:31:02,2020-05-06 11:43:20,2404 091283,2001-01-01,Зохраб,...,False,False,True,False,False,True,NaN,NaN,NaN,urgency_#1
172840,1224750199,30648451,79874514079,5ebbfe5abb03f14ccb5e8f97,Активирован,2020-05-13 17:06:12,2020-05-07 18:26:17,7418 006313,2014-01-01,Ваген,...,False,False,True,False,False,True,NaN,NaN,NaN,urgency_#1
172841,1235750263,30661524,79608382036,5eb9859553d486238a7b6f17,Активирован,2020-05-11 20:08:02,2020-05-11 19:59:14,6332 604518,2017-01-01,Самир,...,False,False,True,False,False,True,NaN,NaN,NaN,urgency_#1
172842,1245500023,30673592,79376559118,5ebe99e953d48674870607db,Активирован,2020-05-15 16:36:03,2020-05-15 06:43:10,6324 135333,2015-01-01,Сирожиддин,...,False,False,True,False,False,True,NaN,NaN,NaN,urgency_#1


In [24]:
# EXPORTING ONLY USERS THAT WEREN'T CHECKED BY S&S AND HAVE ANY ERROR
no_SS_check_errors = df_rus.loc[((df_rus.passport == False)|(df_rus.drivers == False))&(df_rus.any_error == True)]
no_SS_check_errors.to_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/data_quality_results/all_docs/no_S&S_check_v4.csv')

In [4]:
# AGGREGATING ERROR STATISTICS

# replacing NA with FALSE
for i in ['passport', 'drivers',  'selfie']:
    df[i] = df[i].fillna(False)
    
# Aggregating error statistics
df_psp = df_rus[['user_id', 'passport', 'passport_error']]
df_lcns = df_rus[['user_id', 'drivers', 'driving_license_error']]
df_any = df_rus[['user_id', 'passport', 'drivers', 'any_error']]
pvt_psp = pd.pivot_table(df_psp, index = ['passport', 'passport_error'], aggfunc = 'count')
pvt_lcns = pd.pivot_table(df_lcns, index = ['drivers', 'driving_license_error'], aggfunc = 'count')
pvt_any = pd.pivot_table(df_any, index = ['passport','drivers', 'any_error'], aggfunc = 'count')
pvt_psp.to_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/data_quality_results/passport/passport_data_quality_results_pvt_active_users.csv')
pvt_lcns.to_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/data_quality_results/drivers_license/license_data_quality_results_pvt_active_users.csv')
pvt_any.to_csv('C:/Users/sgulbin/Work/Analysis/DataQualityAnalysis/data_quality_results/all_docs/all_docs_data_quality_results_pvt_active_users.csv')
print(pvt_any)

                            user_id
passport drivers any_error         
False    False   False        22171
                 True         31845
         True    False          112
                 True            19
True     False   False        28034
                 True          5798
         True    False        71706
                 True        492440


In [102]:
# TESTING REGULAR EXPRESSIONS

# test = df['License']
# test['error'] = (test['License'].str.replace(' ','')).str.contains(pat = '[a-zA-Z]')
l1 = '1234567890'
l2 = '12АУ567890'
l3 = '12АУ№56789'
l4 = '№123456789'
dc1 = '770-123'
dc2 = '770--123'
dc3 = '770 123'
pat = re.compile('\w{10}')
pat2 = re.compile('\d\d\d-\d\d\d')
for i in [l1,l2,l3,l4]:
    print(i,bool(pat.match(i)) == False)
# for i in [dc1,dc2,dc3]:
#     print(i, bool(pat2.match(i))

1234567890 False
12АУ567890 False
12АУ№56789 True
№123456789 True
